In [1]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

In [2]:
!pip install datasets boto3 einops

In [3]:
%%writefile ~/.passwd-s3fs
# lol

Overwriting /root/.passwd-s3fs


In [4]:
!chmod 600 ~/.passwd-s3fs
!apt install s3fs
!mkdir /s3
!s3fs baj40ja0abjabucketihvp /s3
!ls /s3/ihvp

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
s3fs is already the newest version (1.90-1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
mkdir: cannot create directory ‘/s3’: File exists
s3fs: MOUNTPOINT directory /s3 is not empty. if you are sure this is safe, can use the 'nonempty' mount option.
TinyStories_ihvp_1M.pt	TinyStories_ihvp_tokenwise_33000000.pt


In [5]:
!cp /s3/ihvp/TinyStories_ihvp_1M.pt /content/

In [6]:
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
import torch

# Initialize the tokenizer
# tokenizer.pad_token = tokenizer.eos_token

# https://huggingface.co/datasets/Skylion007/openwebtext
from datasets import load_dataset

class TinyStoriesDataset(Dataset): # IterableDataset
    def __init__(self, split, tokenizer, path='roneneldan/TinyStories', block_size=1024, seed=42, take=10000, skip=0):
        self.split = split
        self.block_size = block_size
        self.tokenizer = tokenizer
        self.dataset = load_dataset(path, split=split, streaming=False) #.shuffle(seed=seed) #.skip(skip).take(take).with_format('torch')

    def __iter__(self):
        for item in self.dataset:
            text = item['text']
            encoding = self.tokenizer.encode_plus(text, return_tensors='pt', add_special_tokens=True, padding='max_length', truncation=True, max_length=self.block_size)
            input_ids = encoding['input_ids'].squeeze()

            # Shift the input_ids and attention_mask to the right and pad
            labels = input_ids.clone()
            labels[:-1] = input_ids[1:]
            labels[-1] = self.tokenizer.pad_token_id

            yield {'input_ids': input_ids}, {'input_ids': labels}

    def __getitem__(self, index):
        item = self.dataset[index]
        text = item['text']
        encoding = self.tokenizer.encode_plus(text, return_tensors='pt', add_special_tokens=True, padding='max_length', truncation=True, max_length=self.block_size)
        input_ids = encoding['input_ids'].squeeze()

        # Shift the input_ids and attention_mask to the right and pad
        labels = input_ids.clone()
        labels[:-1] = input_ids[1:]
        labels[-1] = self.tokenizer.pad_token_id

        return {'input_ids': input_ids}, {'input_ids': labels}

    def __len__(self):
      return len(self.dataset)

    def get_vocab_size(self):
        return self.tokenizer.vocab_size

    def get_block_size(self):
        return self.block_size


In [12]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch import nn

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
tokenizer.pad_token = tokenizer.eos_token
vocab_size = tokenizer.vocab_size

model = AutoModelForCausalLM.from_pretrained("roneneldan/TinyStories-1M")

torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

model.to(device)

GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 64)
    (wpe): Embedding(2048, 64)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-7): 8 x GPTNeoBlock(
        (ln_1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=64, out_features=64, bias=False)
            (v_proj): Linear(in_features=64, out_features=64, bias=False)
            (q_proj): Linear(in_features=64, out_features=64, bias=False)
            (out_proj): Linear(in_features=64, out_features=64, bias=True)
          )
        )
        (ln_2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=64, out_features=256, bias=True)
          (c_proj): Linear(in_features=256, out_

In [17]:
# 1. get a_l-1, use forward hook to save input to a layer l during the forward pass
layer_inputs = {}

# TODO something going on with the shapes
# ONLY ON INPUT TO MLP (batch_size, seq_len, input_dim)
def forward_hook_fn(module, inputs):
    if isinstance(module, nn.Linear):
        print(inputs[0].shape)
        layer_inputs[module] = torch.cat([
                inputs[0],
                torch.ones((inputs[0].shape[0], inputs[0].shape[1], 1)).to(inputs[0].device),
        ], dim=-1).clone().detach()

# 2. get grad_loss, gradients of loss wrt output of linear transformation W_l a_l-1
#    using a backward hook on the linear layer that saves the gradient wrt the linear layer's output

layer_grads = {}

# ONLY ON FIRST LINEAR LAYER self.linear(a_l_minus_1)
def back_hook_fn(module, grad_input, grad_output):
    if isinstance(module, nn.Linear):
        layer_grads[module] = grad_output[0].clone().detach()

linear_layers = []
for name, module in model.named_modules():
    if isinstance(module, nn.Linear) and 'mlp' in name: # out_proj ???
        # grab linear layers everytime
        linear_layers.append(module)

linear_layers = linear_layers[:-1]  # remove output token logits layer from calculations

for _, module in model.named_modules():
    # TODO register miniGPT MLP, first linear layer???
    if _.split('.')[-1] == 'mlp':
      print(_, module)
      # module.register_forward_hook(forward_hook_fn)
    if isinstance(module, nn.Linear) and _.split('.')[-1] == 'c_fc':
      print(_, module)
    #     module.register_full_backward_hook(back_hook_fn)

transformer.h.0.mlp GPTNeoMLP(
  (c_fc): Linear(in_features=64, out_features=256, bias=True)
  (c_proj): Linear(in_features=256, out_features=64, bias=True)
  (act): NewGELUActivation()
  (dropout): Dropout(p=0.0, inplace=False)
)
transformer.h.0.mlp.c_fc Linear(in_features=64, out_features=256, bias=True)
transformer.h.1.mlp GPTNeoMLP(
  (c_fc): Linear(in_features=64, out_features=256, bias=True)
  (c_proj): Linear(in_features=256, out_features=64, bias=True)
  (act): NewGELUActivation()
  (dropout): Dropout(p=0.0, inplace=False)
)
transformer.h.1.mlp.c_fc Linear(in_features=64, out_features=256, bias=True)
transformer.h.2.mlp GPTNeoMLP(
  (c_fc): Linear(in_features=64, out_features=256, bias=True)
  (c_proj): Linear(in_features=256, out_features=64, bias=True)
  (act): NewGELUActivation()
  (dropout): Dropout(p=0.0, inplace=False)
)
transformer.h.2.mlp.c_fc Linear(in_features=64, out_features=256, bias=True)
transformer.h.3.mlp GPTNeoMLP(
  (c_fc): Linear(in_features=64, out_features

In [8]:
model

GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 64)
    (wpe): Embedding(2048, 64)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-7): 8 x GPTNeoBlock(
        (ln_1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=64, out_features=64, bias=False)
            (v_proj): Linear(in_features=64, out_features=64, bias=False)
            (q_proj): Linear(in_features=64, out_features=64, bias=False)
            (out_proj): Linear(in_features=64, out_features=64, bias=True)
          )
        )
        (ln_2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=64, out_features=256, bias=True)
          (c_proj): Linear(in_features=256, out_

In [9]:
def compute_grads(model: nn.Module, train_dataset: DataLoader):

    grads = [[] for _ in range(len(linear_layers))]
    for X, Y in train_dataset:
        model.zero_grad()

        x_ids = X['input_ids'].to(device)
        y_ids = Y['input_ids'].to(device)

        if len(x_ids.shape) == 3:
            x_ids = x_ids.squeeze(1)
            y_ids = y_ids.squeeze(1)

        if len(x_ids.shape) == 1:
            x_ids = x_ids.unsqueeze(0)
            y_ids = y_ids.unsqueeze(0)

        output = model(x_ids, labels=y_ids)
        logits, loss = output['logits'], output['loss']

        loss.backward()
        for i, module in enumerate(linear_layers):
            w_grad = module.weight.grad
            if module.bias is not None:
                b_grad = module.bias.grad.unsqueeze(-1)
                full_grad = torch.cat([w_grad, b_grad], dim=-1)
            else:
                full_grad = torch.cat(
                    [w_grad, torch.zeros([w_grad.shape[0], 1])],
                    dim=-1
                )
            # r = full_grad.view(-1)
            # tokenwise_grads = []
            # for i, _ in enumerate(x_ids[0]):
            #   # print(r[i])
            #   tokenwise_grads.append(r[i])
            grads[i].append(full_grad)
            # print(len(tokenwise_grads))
            # token_stacked = torch.cat(tokenwise_grads, dim=-1)
            # print(token_stacked.shape)

    return grads

In [10]:
prompt = "Once upon a time, there was a girl name Alice. Alice enjoyed many things. One of the things Alice enjoyed was playing outside."
# prompt_enc = tokenizer.encode(prompt, return_tensors='pt').to(device)
# out = model.generate(prompt_enc, max_length=300)
# print(tokenizer.decode(out[0]))
# z_m_ids = torch.nn.functional.pad(out[0], (0, 2048 - len(out[0])), value=50256)

# z_m_label = z_m_ids.clone()
# z_m_label[:-1] = z_m_ids[1:]
# z_m_label[-1] = tokenizer.pad_token_id

# z_m = [({'input_ids': z_m_ids}, {'input_ids': z_m_label})]

prompts = [
    prompt,
    """Emily and Benjamin were exploring the woods behind their house when they stumbled upon an old, weathered book. It was dusty and had a mysterious aura about it, sparking their curiosity instantly. Benjamin turned to Emily, "What should we do with the book?" Emily pondered for a moment, then declared, "Let's take it to Grandma and Grandpa!" With that, they dashed off, their hearts pounding with anticipation, eager to unveil their discovery. They burst into their grandparents' home, exclaiming, "Grandma, Grandpa! Look what we found!" Their grandparents were taken aback and inquired, "Where did you find this old book?""",
    "Once upon a time, there was a small child named Lily. She enjoyed playing with her dolls and observing the butterflies in her garden. Lily had a large, fluffy bunny that she adored immensely. One day, Lily's bunny seemed gloomy. Lily couldn't understand why her bunny was feeling down.",
    "The sun was beaming down as Peter emerged from his small cottage. He was a naive boy who took great pleasure in frolicking and playing games. He had an old rug that he used as a cape, often pretending to be a knight. Peter always longed for a playmate, but there was no one in sight. With a sigh, he resolved to seek out a companion. As he ventured out of the village, he overheard two voices, one of which was distinctly familiar. It was the voice of his neighbor, elderly Mrs. Smith. Peter crept closer, concealing himself behind a tree and peered through the leaves. Mrs. Smith and a stranger were in conversation, and the stranger appeared to be upset about something. Suddenly, the stranger raised his voice and advanced towards Mrs. Smith.",
    """Frank the frog was very troubled. His fountain had broken and he was sure no one could fix it. One day Frank's best friend Sam the snail came to visit him. Sam asked, "What's wrong Frank?" Frank said sadly, "My fountain is broken and I can't fix it." But Sam had a plan! He said, "If you come with me I might know someone who can help you fix your fountain." So Frank and Sam went out into the woods, looking for help. And soon they found Snail Bob!""",

]

def process_prompts(prompts):
  z_m = []
  for prompt in prompts:
    prompt_enc = tokenizer.encode(prompt, return_tensors='pt').to(device)
    out = model.generate(prompt_enc, max_length=1000)
    print(tokenizer.decode(out[0]))
    z_m_ids = torch.nn.functional.pad(out[0], (0, 2048 - len(out[0])), value=50256)
    z_m_label = z_m_ids.clone()
    z_m_label[:-1] = z_m_ids[1:]
    z_m_label[-1] = tokenizer.pad_token_id

    z_m.append(({'input_ids': z_m_ids}, {'input_ids': z_m_label}))
  return z_m

z_m = process_prompts(prompts)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


TypeError: ignored

In [ ]:
import torch
ihvp = torch.load(f'/content/TinyStories_ihvp_1M.pt')

In [ ]:
ihvp.shape

In [ ]:
train_dataset = TinyStoriesDataset('train', tokenizer, block_size=2048, path='roneneldan/TinyStories')

In [ ]:
topk = 5

all_top_training_samples = []
all_top_influences = []
all_top_token_influences = []

test_dataloader = DataLoader(z_m, batch_size=1)

for query, compl in z_m:
    # torch.Tensor(list(filter(lambda x: x != 50256, query['input_ids'].tolist()))).long()
    print(ihvp.shape)
    print(query)
    grads = compute_grads(model, [(query, compl)])
    print("grads[0]", grads[0][0].view(-1).shape)
    print(len(grads))
    for l in grads:
        print("token_query_grad", l[0].view(-1).shape)
    query_grad = torch.cat(
        [q[0].view(-1) for q in grads]
    )
    print("query_grad", query_grad.shape)
    top_influences = -1 * torch.einsum("ij,j->i", ihvp, query_grad)
    print("top_influences", top_influences.shape)

    top_influences, top_samples = torch.topk(top_influences, topk)
    print(top_samples)
    all_top_training_samples.append(top_samples)
    all_top_influences.append(top_influences)
    all_top_token_influences.append(1)

top_influence_sentences = []
for k, (top_samples, top_influences) in enumerate(
        zip(all_top_training_samples, all_top_influences)
    ):
    influence_sentences = []
    for s, i in zip(top_samples, top_influences):
        s = s.item()
        sample = f"{tokenizer.decode(train_dataset[s][0]['input_ids'])}"
        influence_sentences.append({"sample": sample, "influence": i.item()})

    top_influence_sentences.append({"samples": influence_sentences, "query": tokenizer.decode(z_m[k][0]['input_ids'])})


In [ ]:
import json

with open('/content/influences.json', 'w') as outfile:
    json.dump({"influences": top_influence_sentences}, outfile,indent=4)

json.dumps({"influences": top_influence_sentences})

# TODO ensure we are doing this per token when using real data
for i, (top_samples, top_influences, top_token_influences) in enumerate(
        zip(all_top_training_samples, all_top_influences, all_top_token_influences)
    ):
        print(f"Query: {tokenizer.decode(z_m[i][0]['input_ids'])}")
        print(f"Top {topk} training samples and their influences:")
        for s, i in zip(top_samples, top_influences):
            s = s.item()
            sample = f"{tokenizer.decode(train_dataset[s][0]['input_ids'])}"
            print(
                f"###\n{sample}\n Influence: {i}\n"
            )

In [ ]:
import torch as t
import einops
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.utils.data import Dataset
import string
from influence_functions_transformer import influence, InfluenceCalculable
from random import sample

d_model = 16
n_heads = 2
d_mlp = 32
n_layers = 2
vocab_size = 128
dataset_length = 200
sequence_length = 2048
lr = 0.001
n_epochs = 5000

def dataset_sample(dataset, n_samples):
    indices = sample(range(len(dataset)), n_samples)
    return [dataset[i] for i in indices]

def autoregressive_loss(output, target):
    output = einops.rearrange(output, "b s v -> (b s) v")
    target = einops.rearrange(target, "b s -> (b s)")
    loss = t.nn.functional.cross_entropy(output, target)
    return loss


class CharPredictDataset(Dataset):
    def __init__(self, length, seq_length):
        self.data = self._generate_data(length)
        self.seq_length = seq_length

    def _generate_data(self, length):
        alphabets = string.ascii_lowercase
        numbers = [str(i % 10) for i in range(length // 2)]
        return "".join(
            [alphabets[i % len(alphabets)] + numbers[i] for i in range(length // 2)]
        )

    def __len__(self):
        return len(self.data) - self.seq_length

    def __getitem__(self, idx):
        source_seq = self.data[idx : idx + self.seq_length]
        return t.tensor([ord(c) for c in source_seq[:-1]], dtype=t.long), t.tensor(
            [ord(c) for c in source_seq[1:]], dtype=t.long
        )


class MultiHeadMaskedAttention(t.nn.Module):
    def __init__(self, d_model, n_heads):
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.d_head = d_model // n_heads
        self.q_proj = t.nn.Linear(d_model, d_model)
        self.k_proj = t.nn.Linear(d_model, d_model)
        self.v_proj = t.nn.Linear(d_model, d_model)
        self.out_proj = t.nn.Linear(d_model, d_model)

    def forward(self, X, mask=None):
        Q = einops.rearrange(self.q_proj(X), "b s (h d) -> b h s d", h=self.n_heads)
        K = einops.rearrange(self.k_proj(X), "b s (h d) -> b h s d", h=self.n_heads)
        V = einops.rearrange(self.v_proj(X), "b s (h d) -> b h s d", h=self.n_heads)

        # Compute the scaled dot-product attention
        QK = t.einsum("b h i d, b h j d -> b h i j", Q, K)
        QK = QK / t.sqrt(t.tensor(self.d_head))
        if mask is not None:
            QK = QK.masked_fill(mask, -1e9)
        QK = t.nn.functional.softmax(QK, dim=-1)

        # Compute the output
        Y = t.einsum("b h i j, b h j d -> b h i d", QK, V)
        Y = einops.rearrange(Y, "b h s d -> b s (h d)")

        # Apply the output projection
        Y = self.out_proj(Y)

        return Y


class MLPBlock(InfluenceCalculable, t.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.linear = t.nn.Linear(input_dim, hidden_dim)
        self.relu = t.nn.ReLU()
        self.linear2 = t.nn.Linear(hidden_dim, output_dim)
        self.input = None

        # Save gradient of loss wrt output of linear layer (Ds_l, where s_l = self.linear(a_l_minus_1))
        def hook_fn(module, grad_input, grad_output):
            self.d_s_l = grad_output[0]

        self.linear.register_full_backward_hook(hook_fn)

    def forward(self, x):
        self.input = x
        x = self.linear(x)
        x = self.relu(x)
        x = self.linear2(x)
        return x

    def get_a_l_minus_1(self):
        # Return the input to the linear layer as a homogenous vector (batch_size, seq_len, input_dim + 1)
        print(self.input.shape)
        return (
            t.cat(
                [
                    self.input,
                    t.ones((self.input.shape[0], self.input.shape[1], 1)).to(
                        self.input.device
                    ),
                ],
                dim=-1,
            )
            .clone()
            .detach()
        )

    def get_d_s_l(self):
        # Return the gradient of the loss wrt the output of the linear layer
        return self.d_s_l.clone().detach()

    def get_dims(self):
        # Return the dimensions of the weights - (output_dim, input_dim)
        return self.linear.weight.shape

    def get_d_w_l(self):
        # Return the gradient of the loss wrt the weights
        w_grad = self.linear.weight.grad
        b_grad = self.linear.bias.grad.unsqueeze(-1)
        full_grad = t.cat([w_grad, b_grad], dim=-1)
        return full_grad.clone().detach()


class TransformerBlock(t.nn.Module):
    def __init__(self, d_model, n_heads, d_mlp):
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.d_mlp = d_mlp
        self.attn = MultiHeadMaskedAttention(d_model, n_heads)
        self.mlp = MLPBlock(d_model, d_mlp, d_model)
        self.layer_norm1 = t.nn.LayerNorm(d_model)
        self.layer_norm2 = t.nn.LayerNorm(d_model)

    def forward(self, X, mask=None):
        attn_output = self.attn(X, mask)
        X = self.layer_norm1(X + attn_output)
        mlp_output = self.mlp(X)
        Y = self.layer_norm2(X + mlp_output)
        return Y


class DecoderTransformer(t.nn.Module):
    def __init__(self, d_model, n_heads, d_mlp, n_layers, vocab_size, max_seq_len):
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.d_mlp = d_mlp
        self.n_layers = n_layers
        self.vocab_size = vocab_size
        self.embed_input = t.nn.Embedding(vocab_size, d_model)
        self.blocks = t.nn.ModuleList(
            [TransformerBlock(d_model, n_heads, d_mlp) for _ in range(n_layers)]
        )
        self.out_proj = t.nn.Linear(d_model, vocab_size)
        self.position_embeddings = t.nn.Embedding(max_seq_len, d_model)
        self.device = t.device("cuda" if t.cuda.is_available() else "cpu")

    def forward(self, X):
        seq_len = X.size(-1)
        mask = t.triu(t.ones(seq_len, seq_len), diagonal=1).bool().to(self.device)
        X = self.embed_input(X)
        positions = t.arange(0, seq_len, device=X.device).unsqueeze(0)
        X = X + self.position_embeddings(positions)

        for block in self.blocks:
            X = block(X, mask)
        Y = self.out_proj(X)
        return Y


def train_loop(model, data_loader, optimizer, num_epochs):
    model.train()
    device = t.device("cuda" if t.cuda.is_available() else "cpu")
    model.to(device)
    print_every = num_epochs // 50
    print_every = 1 if print_every == 0 else print_every
    for epoch in range(num_epochs):
        total_loss = 0
        for model_input, target in data_loader:
            model_input, target = model_input.to(device), target.to(device)
            output = model(model_input)
            loss = autoregressive_loss(output, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(data_loader)
        if (epoch + 1) % print_every == 0:
            print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}")


def train_char_predict():
    small_transformer = DecoderTransformer(
        d_model=d_model,
        n_heads=n_heads,
        d_mlp=d_mlp,
        n_layers=n_layers,
        vocab_size=vocab_size,
        max_seq_len=sequence_length,
    )
    # dataset = CharPredictDataset(length=dataset_length, seq_length=sequence_length)
    dataset = TinyStoriesDataset('train', tokenizer, block_size=2048, path='roneneldan/TinyStories')

    data_loader = DataLoader(dataset, batch_size=32, shuffle=True)
    optimizer = Adam(small_transformer.parameters(), lr=lr)
    train_loop(small_transformer, data_loader, optimizer, num_epochs=n_epochs)
    t.save(small_transformer.state_dict(), "small_transformer.pth")


def calc_influence(model_path):
    device = t.device("cuda" if t.cuda.is_available() else "cpu")
    # train_dataset = CharPredictDataset(
    #     length=dataset_length, seq_length=sequence_length
    # )
    train_dataset = TinyStoriesDataset('train', tokenizer, block_size=2048, path='roneneldan/TinyStories')
    model = DecoderTransformer(
        d_model, n_heads, d_mlp, n_layers, vocab_size, sequence_length
    )
    model.load_state_dict(t.load(model_path))
    model.to(device)
    model.eval()

    topk = 10
    queries = dataset_sample(train_dataset, 5)
    gradient_fitting_data = dataset_sample(train_dataset, 100)
    search_data = dataset_sample(train_dataset, 100)

    all_top_training_samples, all_top_influences = influence(
        model,
        [b.mlp for b in model.blocks],
        queries,
        gradient_fitting_data,
        search_data,
        topk,
        device,
    )

    def decode(token_ids):
        try:
            return "".join([chr(i) for i in token_ids])
        except:
            return chr(token_ids)

    for i, (top_samples, top_influences) in enumerate(
        zip(all_top_training_samples, all_top_influences)
    ):
        print(f"Query: {decode(queries[i][0])[0]}{decode(queries[i][1])}")
        print(f"Top {topk} training samples and their influences:")
        for s, i in zip(top_samples, top_influences):
            s = s.item()
            print(
                f"{decode(search_data[s][0])[0]}{decode(search_data[s][1])} Influence: {i}"
            )


def run_model(model_path):
    model = DecoderTransformer(
        d_model, n_heads, d_mlp, n_layers, vocab_size, sequence_length
    )
    model.load_state_dict(t.load(model_path))

    model.eval()
    device = t.device("cuda" if t.cuda.is_available() else "cpu")
    model.to(device)

    while True:
        user_input = input("Enter a string: ")
        if user_input == "exit":
            return
        if len(user_input) > (sequence_length - 1):
            user_input = user_input[-(sequence_length - 1) :]
        token_ids = t.tensor([[ord(c) for c in user_input]], dtype=t.long).to(device)
        model_output = model(token_ids)
        last_token = model_output[0, -1, :]
        topk = t.topk(last_token, 1)
        topk_tokens = [chr(int(i)) for i in topk.indices.tolist()]
        print(topk_tokens[0])


if __name__ == "__main__":
    train_char_predict()
    # run_model("small_transformer.pth")
    # calc_influence("/content/small_transformer.pth")
